In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Load dataset

df_train = pd.read_csv('data/Train.csv')
df_test = pd.read_csv('data/Test.csv')
df_sample_submission = pd.read_csv('data/SampleSubmission.csv')

In [3]:
# Creating subsets of locations
#location_groups = df_train.groupby('Place_ID')
#type("location_groups")
#for location, sub_df in location_groups:
    # Sort each subset DataFrame by the 'Date' column in ascending order
#   sub_df = sub_df.sort_values(by='Date', ascending=True)
#  print(f"{location}:{sub_df}")


# Convert 'Date' to datetime if not already done
df_train['Date'] = pd.to_datetime(df_train['Date'])

# Create empty lists to store train and test DataFrames
train_dfs = []
test_dfs = []

# Creating subsets of locations
location_groups = df_train.groupby('Place_ID')

# Filter out groups that have fewer than 13 unique dates
filtered_groups = {
    location: sub_df for location, sub_df in location_groups 
    if sub_df['Date'].nunique() >= 13
}

# Debug: Print number of filtered groups
print(f"Number of filtered groups (locations with >=13 unique dates): {len(filtered_groups)}")

# Iterate through each filtered group
for location, sub_df in filtered_groups.items():
    # Sort each subset DataFrame by the 'Date' column in ascending order
    sub_df = sub_df.sort_values(by='Date', ascending=True)

    # Debug: Check the number of rows in the sorted sub_df
    print(f"Location: {location}, Number of rows: {len(sub_df)}")
    
    # Select the last three rows for the test set
    test_df = sub_df.tail(3)
    # Select the remaining rows for the train set
    train_df = sub_df.iloc[:-3]

     # Create a lag-1 variable for 'target' only in the training set
    train_df['lag_1'] = train_df['target'].shift(1)

    # Drop rows with NaN values in 'lag_1' (due to the shift)
    train_df = train_df.dropna(subset=['lag_1'])

    # Debug: Verify the number of rows in train and test sets for this location
    print(f"Location: {location}, Train Rows: {len(train_df)}, Test Rows: {len(test_df)}")

    # Append the split dataframes to their respective lists
    train_dfs.append(train_df)
    test_dfs.append(test_df)

# Concatenate all the train and test DataFrames into separate DataFrames
df_train_split = pd.concat(train_dfs, ignore_index=True)
df_test_split = pd.concat(test_dfs, ignore_index=True)

# Display or use the resulting DataFrames
print("Training set:")
print(df_train_split.head())  # Display the first few rows for verification
print("\nTest set:")
print(df_test_split.head())   # Display the first few rows for verification

# Optional: Check the number of rows in the test set for each location to verify correctness
print("\nNumber of rows per Place_ID in the test set:")
print(df_test_split['Place_ID'].value_counts())

Number of filtered groups (locations with >=13 unique dates): 337
Location: 010Q650, Number of rows: 94
Location: 010Q650, Train Rows: 90, Test Rows: 3
Location: 05EC30X, Number of rows: 90
Location: 05EC30X, Train Rows: 86, Test Rows: 3
Location: 0DPWHX8, Number of rows: 94
Location: 0DPWHX8, Train Rows: 90, Test Rows: 3
Location: 0GBXTHY, Number of rows: 94
Location: 0GBXTHY, Train Rows: 90, Test Rows: 3
Location: 0HYPV1N, Number of rows: 94
Location: 0HYPV1N, Train Rows: 90, Test Rows: 3
Location: 0I2XREH, Number of rows: 65
Location: 0I2XREH, Train Rows: 61, Test Rows: 3
Location: 0KV6RJ1, Number of rows: 94
Location: 0KV6RJ1, Train Rows: 90, Test Rows: 3
Location: 0MGEY68, Number of rows: 94
Location: 0MGEY68, Train Rows: 90, Test Rows: 3
Location: 0O9R6AD, Number of rows: 92
Location: 0O9R6AD, Train Rows: 88, Test Rows: 3
Location: 0PP73FZ, Number of rows: 94
Location: 0PP73FZ, Train Rows: 90, Test Rows: 3
Location: 0RYZQUU, Number of rows: 91
Location: 0RYZQUU, Train Rows: 87, T

In [4]:
# Check if 'lag_1' exists in the training dataset and move it to the end
if 'lag_1' in df_train_split.columns:
    # Get all columns except 'lag_1'
    cols = [col for col in df_train_split.columns if col != 'lag_1']
    # Add 'lag_1' as the last column
    df_train_split = df_train_split[cols + ['lag_1']]
    print("'lag_1' has been moved to the last position in the training dataset.")
else:
    print("'lag_1' does not exist in the training dataset.")

'lag_1' has been moved to the last position in the training dataset.


In [5]:
# Define the feature columns to check for missing values
feature_columns = [
    'temperature_2m_above_ground', 
    'precipitable_water_entire_atmosphere',  
    'Place_ID',
    'lag_1'
]

# Drop rows with missing values in any of the specified feature columns
df_train_split.dropna(subset=feature_columns)

# Display the cleaned DataFrame
print("DataFrame after dropping rows with missing values in feature columns:")
print(df_train_split)


DataFrame after dropping rows with missing values in feature columns:
            Place_ID X Date       Date Place_ID  target  target_min   
0      010Q650 X 2020-01-03 2020-01-03  010Q650    39.0        25.0  \
1      010Q650 X 2020-01-04 2020-01-04  010Q650    24.0         8.0   
2      010Q650 X 2020-01-05 2020-01-05  010Q650    49.0        10.0   
3      010Q650 X 2020-01-06 2020-01-06  010Q650    21.0         9.0   
4      010Q650 X 2020-01-07 2020-01-07  010Q650    28.0        10.0   
...                     ...        ...      ...     ...         ...   
29182  YWSFY6Q X 2020-03-12 2020-03-12  YWSFY6Q    56.0        23.0   
29183  YWSFY6Q X 2020-03-13 2020-03-13  YWSFY6Q    45.0        23.0   
29184  YWSFY6Q X 2020-03-14 2020-03-14  YWSFY6Q    22.0        12.0   
29185  YWSFY6Q X 2020-03-15 2020-03-15  YWSFY6Q    22.0        14.0   
29186  YWSFY6Q X 2020-03-16 2020-03-16  YWSFY6Q    53.0        30.0   

       target_max  target_variance  target_count   
0            63.0        

In [6]:
# Function to find missing dates for each Place_ID
def check_missing_dates(df):
    missing_info = []
    
    # Group by Place_ID
    grouped = df.groupby('Place_ID')
    
    for place_id, group in grouped:
        # Sort the group by 'Date'
        group = group.sort_values(by='Date')
        
        # Get min and max date for the current Place_ID
        min_date = group['Date'].min()
        max_date = group['Date'].max()
        
        # Generate the full range of dates
        full_date_range = pd.date_range(start=min_date, end=max_date, freq='D')
        
        # Find actual dates in the group
        actual_dates = group['Date'].unique()
        
        # Identify missing dates
        missing_dates = set(full_date_range) - set(actual_dates)
        
        # Append result for this Place_ID
        missing_info.append({
            'Place_ID': place_id,
            'min_date': min_date,
            'max_date': max_date,
            'expected_dates': len(full_date_range),
            'actual_dates': len(actual_dates),
            'missing_dates_count': len(missing_dates),
            'missing_dates': sorted(missing_dates)
        })
    
    return pd.DataFrame(missing_info)

# Run the function to check missing dates
missing_dates_df = check_missing_dates(df_train_split)

# Display the summary of missing dates
print(missing_dates_df)

# Check rows where missing dates exist
missing_rows = missing_dates_df[missing_dates_df['missing_dates_count'] > 0]
print("Locations with missing dates:")
print(missing_rows)

    Place_ID   min_date   max_date  expected_dates  actual_dates   
0    010Q650 2020-01-03 2020-04-01              90            90  \
1    05EC30X 2020-01-03 2020-04-01              90            86   
2    0DPWHX8 2020-01-03 2020-04-01              90            90   
3    0GBXTHY 2020-01-03 2020-04-01              90            90   
4    0HYPV1N 2020-01-03 2020-04-01              90            90   
..       ...        ...        ...             ...           ...   
332  YLLOKEY 2020-01-03 2020-04-01              90            90   
333  YLZOBFW 2020-01-03 2020-04-01              90            90   
334  YPXSK14 2020-01-03 2020-04-01              90            90   
335  YSIXKFZ 2020-01-03 2020-04-01              90            90   
336  YWSFY6Q 2020-01-03 2020-03-16              74            74   

     missing_dates_count                                      missing_dates  
0                      0                                                 []  
1                      4  [

In [7]:
# Step 1: Identify Place_IDs with missing dates
place_ids_with_missing_dates = missing_dates_df[missing_dates_df['missing_dates_count'] > 0]['Place_ID']

# Step 2: Filter out these Place_IDs from your original DataFrames
df_train_split_no_missing = df_train_split[~df_train_split['Place_ID'].isin(place_ids_with_missing_dates)]

# Display the filtered DataFrames
print("Filtered df_train_split without missing dates:")
print(df_train_split_no_missing)


Filtered df_train_split without missing dates:
            Place_ID X Date       Date Place_ID  target  target_min   
0      010Q650 X 2020-01-03 2020-01-03  010Q650    39.0        25.0  \
1      010Q650 X 2020-01-04 2020-01-04  010Q650    24.0         8.0   
2      010Q650 X 2020-01-05 2020-01-05  010Q650    49.0        10.0   
3      010Q650 X 2020-01-06 2020-01-06  010Q650    21.0         9.0   
4      010Q650 X 2020-01-07 2020-01-07  010Q650    28.0        10.0   
...                     ...        ...      ...     ...         ...   
29182  YWSFY6Q X 2020-03-12 2020-03-12  YWSFY6Q    56.0        23.0   
29183  YWSFY6Q X 2020-03-13 2020-03-13  YWSFY6Q    45.0        23.0   
29184  YWSFY6Q X 2020-03-14 2020-03-14  YWSFY6Q    22.0        12.0   
29185  YWSFY6Q X 2020-03-15 2020-03-15  YWSFY6Q    22.0        14.0   
29186  YWSFY6Q X 2020-03-16 2020-03-16  YWSFY6Q    53.0        30.0   

       target_max  target_variance  target_count   
0            63.0          1319.85            91

In [8]:
# Step 1: Identify Place_IDs with missing dates
place_ids_with_missing_dates = missing_dates_df[missing_dates_df['missing_dates_count'] > 0]['Place_ID']

# Step 2: Filter out these Place_IDs from your original DataFrames
df_test_split_no_missing = df_test_split[~df_test_split['Place_ID'].isin(place_ids_with_missing_dates)]

# Display the filtered DataFrames
print("\nFiltered df_test_split without missing dates:")
print(df_test_split_no_missing)


Filtered df_test_split without missing dates:
           Place_ID X Date       Date Place_ID  target  target_min   
0     010Q650 X 2020-04-02 2020-04-02  010Q650    59.0        51.0  \
1     010Q650 X 2020-04-03 2020-04-03  010Q650    61.0        52.0   
2     010Q650 X 2020-04-04 2020-04-04  010Q650    45.0        25.0   
6     0DPWHX8 X 2020-04-02 2020-04-02  0DPWHX8    57.0        51.0   
7     0DPWHX8 X 2020-04-03 2020-04-03  0DPWHX8    60.0        51.0   
...                    ...        ...      ...     ...         ...   
1006  YSIXKFZ X 2020-04-03 2020-04-03  YSIXKFZ    31.0        10.0   
1007  YSIXKFZ X 2020-04-04 2020-04-04  YSIXKFZ    37.0        23.0   
1008  YWSFY6Q X 2020-03-17 2020-03-17  YWSFY6Q    85.0        52.0   
1009  YWSFY6Q X 2020-03-18 2020-03-18  YWSFY6Q   103.0        33.0   
1010  YWSFY6Q X 2020-03-19 2020-03-19  YWSFY6Q    89.0        46.0   

      target_max  target_variance  target_count   
0           73.0           261.12            96  \
1         

In [10]:
# MODEL 1

from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Step 1: Sort data by Place_ID and Date
df_train_split_no_missing = df_train_split_no_missing.sort_values(by=['Place_ID', 'Date'])
df_test_split_no_missing = df_test_split_no_missing.sort_values(by=['Place_ID', 'Date'])


# Convert 'Date' to datetime format if not already done
df_train_split_no_missing['Date'] = pd.to_datetime(df_train_split_no_missing['Date'])
df_test_split_no_missing['Date'] = pd.to_datetime(df_test_split_no_missing['Date'])


# Step 3: Update the feature columns list to include the new features
feature_columns = [
    'temperature_2m_above_ground', 
    'precipitable_water_entire_atmosphere',  
    'Place_ID'
]

feature_columns_lag = feature_columns + ["lag_1"]

# Prepare the training and testing data with the new features
X_train = df_train_split_no_missing[feature_columns_lag]
y_train = df_train_split_no_missing['target']
#X_test = df_test_split[feature_columns]
#y_test = df_test_split['target']

# Encode the 'Place_ID' as it is a categorical feature
label_encoder = LabelEncoder()
X_train['Place_ID'] = label_encoder.fit_transform(X_train['Place_ID'])
#X_test['Place_ID'] = label_encoder.transform(X_test['Place_ID'])

# Define the LightGBM model using the scikit-learn API
model = LGBMRegressor(
    objective='regression',
    learning_rate=0.05,
    n_estimators=1000,
    num_leaves=31,
    max_depth=-1,
    min_data_in_leaf=20,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5,
    random_state=42
)

X_train_new, X_val, y_train_new, y_val = train_test_split(X_train, y_train, random_state=42, test_size=0.2)

# Train the model with early stopping
model.fit(
    X_train_new, y_train_new,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
    callbacks=[early_stopping(stopping_rounds=50), log_evaluation(period=100)],
)


# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
print(f"Root Mean Squared Error (RMSE): {rmse}")


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Auto-choosing col-wise multi-threading, 

In [13]:
# MODEL 1 (for test)

from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Step 1: Sort data by Place_ID and Date
df_train_split_no_missing = df_train_split_no_missing.sort_values(by=['Place_ID', 'Date'])
df_test_split_no_missing = df_test_split_no_missing.sort_values(by=['Place_ID', 'Date'])


# Convert 'Date' to datetime format if not already done
df_train_split_no_missing['Date'] = pd.to_datetime(df_train_split_no_missing['Date'])
df_test_split_no_missing['Date'] = pd.to_datetime(df_test_split_no_missing['Date'])


# Step 3: Update the feature columns list to include the new features
feature_columns = [
    'temperature_2m_above_ground', 
    'precipitable_water_entire_atmosphere',  
    'Place_ID'
]

# Prepare the training and testing data with the new features
X_train = df_train_split_no_missing[feature_columns]
y_train = df_train_split_no_missing['target']
X_test = df_test_split_no_missing[feature_columns]
y_test = df_test_split_no_missing['target']

# Encode the 'Place_ID' as it is a categorical feature
label_encoder = LabelEncoder()
X_train['Place_ID'] = label_encoder.fit_transform(X_train['Place_ID'])
X_test['Place_ID'] = label_encoder.transform(X_test['Place_ID'])

# Define the LightGBM model using the scikit-learn API
model = LGBMRegressor(
    objective='regression',
    learning_rate=0.05,
    n_estimators=1000,
    num_leaves=31,
    max_depth=-1,
    min_data_in_leaf=20,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5,
    random_state=42
)

X_train_new, X_val, y_train_new, y_val = train_test_split(X_train, y_train, random_state=42, test_size=0.2)

# Train the model with early stopping
model.fit(
    X_train_new, y_train_new,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
    callbacks=[early_stopping(stopping_rounds=50), log_evaluation(period=100)],
)


# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
print(f"Root Mean Squared Error (RMSE): {rmse}")


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Auto-choosing col-wise multi-threading, 